### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [3]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [4]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

In [10]:
keywords = ["CacheNodeidentityInfo", "EnableSeparateConfigBase", "ExperimentalClusterDNS",
            "GoogleCloudBucketACL", "KeepLaunchConfigurations", "SpecOverrideFlag", "Spotinst",
            "SpotinstOcean", "SpotinstHybrid", "SpotinstController", "VFSVaultSupport",
            "VPCSkipEnableDNSSupport", "SkipEtcdVersionCheck", "TerraformJSON", "ClusterAddons",
            "KopsControllerStateStore", "APIServerNodes", "UseAddonOperators", "AWSIPv6",
            "TerraformManagedFiles", "AlphaAllowGCE", "AlphaAllowALI"]
# removing "Azure" because it affects more than 100 files

### List files with keywords

#### List all .go files

In [11]:
go_folders = [x[0] for x in os.walk("./kops-master/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./kops-master/vendor/github.com/Azure/go-autorest/autorest/azure/metadata_environment.go
./kops-master/vendor/k8s.io/client-go/tools/leaderelection/resourcelock/configmaplock.go
./kops-master/vendor/github.com/modern-go/reflect2/unsafe_type.go
./kops-master/vendor/github.com/digitalocean/godo/action.go
./kops-master/vendor/github.com/containerd/containerd/remotes/docker/schema1/converter.go
./kops-master/vendor/github.com/Azure/azure-sdk-for-go/services/resources/mgmt/2020-06-01/resources/deployments.go
./kops-master/vendor/sigs.k8s.io/kustomize/kyaml/fn/runtime/starlark/doc.go
./kops-master/vendor/oras.land/oras-go/pkg/context/context.go
./kops-master/vendor/k8s.io/apimachinery/pkg/util/httpstream/httpstream.go
./kops-master/vendor/github.com/miekg/dns/nsecx.go


In [12]:
len(go_files)

8304

#### List the interesting files, containing at least one keyword with the featureflag 

In [17]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0 and "featureflag" in s:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./kops-master/upup/pkg/fi/cloudup/template_functions.go contains :
-> CacheNodeidentityInfo
-> Spotinst
-> APIServerNodes


./kops-master/upup/pkg/fi/cloudup/apply_cluster.go contains :
-> Spotinst
-> SpotinstHybrid
-> KopsControllerStateStore
-> TerraformManagedFiles
-> AlphaAllowGCE
-> AlphaAllowALI


./kops-master/upup/pkg/fi/cloudup/populate_instancegroup_spec.go contains :
-> APIServerNodes


./kops-master/upup/pkg/fi/cloudup/new_cluster.go contains :
-> Spotinst


./kops-master/upup/pkg/fi/cloudup/alitasks/launchconfiguration.go contains :
-> KeepLaunchConfigurations


./kops-master/upup/pkg/fi/cloudup/terraform/target_hcl2.go contains :
-> Spotinst


./kops-master/upup/pkg/fi/cloudup/terraform/target.go contains :
-> TerraformJSON
-> TerraformManagedFiles


./kops-master/upup/pkg/fi/cloudup/bootstrapchannelbuilder/bootstrapchannelbuilder.go contains :
-> Spotinst
-> SpotinstController
-> ClusterAddons
-> UseAddonOperators


./kops-master/upup/pkg/fi/cloudup/awsup/aws_cloud.go co

In [18]:
len(go_file_interests)

29

## Step 3 - Search Feature Toggles dependencies in the AST

In [19]:
for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                #print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./kops-master/upup/pkg/fi/cloudup/template_functions.go 

-> featureflag.Spotinst.Enabled()
-> err == nil
-> featureflag.APIServerNodes.Enabled()
-> featureflag.CacheNodeidentityInfo.Enabled()


File :             ./kops-master/upup/pkg/fi/cloudup/apply_cluster.go 

-> !featureflag.AlphaAllowGCE.Enabled()
-> !featureflag.AlphaAllowALI.Enabled()
-> featureflag.Spotinst.Enabled()
-> featureflag.Spotinst.Enabled()
-> featureflag.SpotinstHybrid.Enabled()
-> featureflag.SpotinstHybrid.Enabled()
-> ok && featureflag.TerraformManagedFiles.Enabled()


File :             ./kops-master/upup/pkg/fi/cloudup/populate_instancegroup_spec.go 

-> ig.IsMaster()
-> ig.Spec.Role == kops.InstanceGroupRoleBastion
-> ig.IsAPIServerOnly() && !featureflag.APIServerNodes.Enabled()


File :             ./kops-master/upup/pkg/fi/cloudup/new_cluster.go 

-> featureflag.Spotinst.Enabled()
-> opt.SpotinstProduct != ""
-> opt.SpotinstOrientation != ""


File :             ./kops-master/upup/pkg/fi

In [16]:
type_nodes

{'comment': 18,
 'package_clause': 1,
 'package': 1,
 'package_identifier': 1,
 '\n': 158,
 'import_declaration': 1,
 'import': 1,
 'import_spec_list': 1,
 '(': 109,
 'import_spec': 13,
 'interpreted_string_literal': 125,
 '"': 250,
 ')': 109,
 'method_declaration': 6,
 'func': 9,
 'parameter_list': 17,
 'parameter_declaration': 11,
 'identifier': 296,
 'pointer_type': 6,
 '*': 7,
 'type_identifier': 26,
 'field_identifier': 146,
 'block': 57,
 '{': 65,
 'var_declaration': 2,
 'var': 2,
 'var_spec': 2,
 'call_expression': 88,
 'selector_expression': 140,
 '.': 140,
 'argument_list': 88,
 'func_literal': 1,
 'assignment_statement': 29,
 'expression_list': 158,
 '=': 29,
 '}': 65,
 'return_statement': 26,
 'return': 26,
 'if_statement': 42,
 'if': 42,
 'short_var_declaration': 30,
 ':=': 34,
 ';': 6,
 'binary_expression': 55,
 '!=': 20,
 'nil': 18,
 ',': 83,
 '||': 2,
 'expression_switch_statement': 4,
 'switch': 4,
 'expression_case': 10,
 'case': 10,
 ':': 18,
 '==': 21,
 'int_literal'

In [14]:
# Pattern matching -> does not work yet

query = GO_LANGUAGE.query("""
(function_definition
  name: (identifier) @function.def)

(call
  function: (identifier) @function.call)
""")

captures = query.captures(tree.root_node)

